In [2]:
!pip install rouge-score
!pip install nltk

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/133.7 kB ? eta -:--:--
     ---------                                30.7/133.7 kB ? eta -:--:--
     --------------------------------       112.6/133.7 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 133.7/133.7 kB 1.1 MB/s eta 0:00:00
                                              0.0/1.5 MB ? eta -:--:--
     ----                                     0.2/1.5 MB 9.0 MB/s eta 0:00:01
     -----------                              0.4/1.5 MB 5.3 MB/s eta 0:00:01
     ---------------                          0.6/1.5 MB 4.7 MB/s eta 0:00:01
     --------------------                     0.8/1.5 MB 4


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from rouge_score import rouge_scorer
import pandas as pd

In [4]:
def evaluate_rouge(golden, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(golden, generated)

In [5]:
generated_docstrings_data = pd.read_csv("function 10 all 3 models.csv")

In [6]:
gpt_docstring_function_10 = generated_docstrings_data.loc[10, "GPT-3.5 Turbo"]
claude_docstring_function_10 = generated_docstrings_data.loc[10, "claude"]
gemini_docstring_function_10 = generated_docstrings_data.loc[10, "Gemini"]

golden_docstring_function_10 = generated_docstrings_data.loc[10, "Golden Docstring"]

In [7]:
print(evaluate_rouge(golden_docstring_function_10, gpt_docstring_function_10))
print(evaluate_rouge(golden_docstring_function_10, claude_docstring_function_10))
print(evaluate_rouge(golden_docstring_function_10, gemini_docstring_function_10))



{'rouge1': Score(precision=0.8108108108108109, recall=0.8571428571428571, fmeasure=0.8333333333333334), 'rouge2': Score(precision=0.6944444444444444, recall=0.7352941176470589, fmeasure=0.7142857142857144), 'rougeL': Score(precision=0.7837837837837838, recall=0.8285714285714286, fmeasure=0.8055555555555555)}
{'rouge1': Score(precision=0.417910447761194, recall=0.8, fmeasure=0.5490196078431372), 'rouge2': Score(precision=0.24242424242424243, recall=0.47058823529411764, fmeasure=0.32), 'rougeL': Score(precision=0.373134328358209, recall=0.7142857142857143, fmeasure=0.49019607843137253)}
{'rouge1': Score(precision=0.8484848484848485, recall=0.8, fmeasure=0.823529411764706), 'rouge2': Score(precision=0.5625, recall=0.5294117647058824, fmeasure=0.5454545454545455), 'rougeL': Score(precision=0.696969696969697, recall=0.6571428571428571, fmeasure=0.676470588235294)}


In [8]:
print(gpt_docstring_function_10)

    Computes the difference between sparse matrices represented as dictionaries.
    Args:
    - lst (list): A list of dictionaries representing sparse matrices.
    Returns:
    - dict: A dictionary representing the result of the subtraction of all sparse matrices in the input list.


In [9]:
print(claude_docstring_function_10)

 """Iteratively computes the difference of sparse matrices stored in dictionaries by subtracting corresponding entries.

Args:
- lst (list): a list of dictionaries, where each dictionary represents a sparse matrix with integer entries. 

Returns: 
- dict: the dictionary representing the difference matrix, with entries corresponding to the subtraction of matrices in lst. Entries not present in all matrices will have the value of the negative entry from the subtracting matrix.
"""


In [10]:
t5 = pd.read_csv("T5 BaseLine docstring generation.csv")

In [11]:
a = t5.loc[10, "T5 BaseLine docstring generation"]

In [12]:
evaluate_rouge(golden_docstring_function_10, a)

{'rouge1': Score(precision=0.8, recall=0.11428571428571428, fmeasure=0.19999999999999998),
 'rouge2': Score(precision=0.5, recall=0.058823529411764705, fmeasure=0.10526315789473684),
 'rougeL': Score(precision=0.8, recall=0.11428571428571428, fmeasure=0.19999999999999998)}

## BLuE

We have decided to use method5 as our smoothing funciton based on the results we've got when comparing to the other options and after reading 'A Systematic Comparison of Smoothing Techniques for Sentence-Level
BLEU' by Boxing Chen and Colin Cherry.
We've chosen it since it is quite intuitive and performs well for our purpouses - emphesizing meaning and recognizing the similarity between phrases even if there are slight variations or shifts in wording. 

In [14]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

chencherry = SmoothingFunction()

def get_bleu_score(ref, candidate):
    return sentence_bleu([ref.split()], candidate.split(), smoothing_function=chencherry.method5)

In [16]:
print(get_bleu_score(golden_docstring_function_10, a))
print(get_bleu_score(golden_docstring_function_10, gpt_docstring_function_10))
print(get_bleu_score(golden_docstring_function_10, claude_docstring_function_10))
print(get_bleu_score(golden_docstring_function_10, gemini_docstring_function_10))

0.0004276919165459266
0.5027541731081724
0.2453357124137627
0.42940530391199083
